# Generate Truncated Data for Robustness Evaluation

## CIFAR-10

In [ ]:
import torchvision
import torchvision.transforms as transforms
import torch
import numpy as np

test_data = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transforms.ToTensor())

### Invert image color

In [ ]:
from PIL import ImageOps
to = transforms.ToTensor()

invert_test_data = []
invert_test_target = []

for image, label in test_data:
    img = transforms.functional.to_pil_image(image)
    img = ImageOps.invert(img)
    
    invert_test_data.append(to(img))
    
    invert_test_target.append((label))

torch.save([invert_test_data, invert_test_target], 'invert_cifar10_test.pt')

### Add gaussian noise

In [ ]:
def add_gaussian_noise(image_tensor, mean=0, var=0.005):

    image = image_tensor.permute(1, 2, 0).numpy()  
    
  
    noise = np.random.normal(mean, var ** 0.5, image.shape)
    out = image + noise
   
    out = np.clip(out, 0.0, 1.0)  
   
    out = torch.from_numpy(np.float32(out)).permute(2, 0, 1)
    
    return out


noisy_test_data = []
noisy_test_target = []


for image, label in test_data:
    
    new_img = add_gaussian_noise(image)
    noisy_test_data.append((new_img))
    noisy_test_target.append((label))
    

torch.save([noisy_test_data, noisy_test_target], 'noise_cifar10_test.pt')

### Add blur effects

In [ ]:
from PIL import ImageFilter

blur_test_data = []
blur_test_target = []
to = transforms.ToTensor()
for image, label in test_data:
    img = transforms.functional.to_pil_image(image)
    img = img.filter(ImageFilter.GaussianBlur(radius=2))
    blur_test_data.append(to(img))
    blur_test_target.append((label))

torch.save([blur_test_data, blur_test_target], 'blur_cifar10_test.pt')

## ImageNet

In [ ]:
import os
import numpy as np
from PIL import Image, ImageFilter, ImageOps
from tqdm import tqdm


def add_noise(image):
    image = np.asarray(image,dtype=np.float32) / 255.
    noise = np.random.normal(0, 0.2, image.shape).astype(dtype=np.float32)
    output = image + noise
    output = np.clip(output, 0, 1)
    output = np.uint8(output * 255)
    output = Image.fromarray(output)
    return output


def blur(image):
    img = image.filter(ImageFilter.GaussianBlur(radius=2))
    return img


def invert(image):
    try:
        img = ImageOps.invert(image)
    except OSError:
        image = image.convert('RGB')
        img = ImageOps.invert(image)
        
    return img


def make_dirs(image_dir, image_class):
    os.makedirs(f'{image_dir}/train/{image_class}', exist_ok=True)
    os.makedirs(f'{image_dir}/val/{image_class}', exist_ok=True)


imagenet_dir = '/data/datasets/ImageNet' # Modify the directory here for your own setting.
imagenet_noise_dir = f'{imagenet_dir}/ImageNet-noise' # Target directory for storing ImageNet with gaussian noise.
imagenet_invert_dir = f'{imagenet_dir}/ImageNet-invert' # Target directory for storing ImageNet with gaussian noise.
imagenet_blur_dir = f'{imagenet_dir}/ImageNet-blur' # Target directory for storing ImageNet with gaussian noise.
image_classes = os.listdir(f'{imagenet_dir}/val')
print("working.....")
for image_class in tqdm(image_classes):
    make_dirs(imagenet_noise_dir, image_class)
    make_dirs(imagenet_invert_dir, image_class)
    make_dirs(imagenet_blur_dir, image_class)
    image_paths = os.listdir(f'{imagenet_dir}/val/{image_class}')
    for path in image_paths:
        image = Image.open(f'{imagenet_dir}/val/{image_class}/{path}')
        image_noise = add_noise(image)
        image_invert = invert(image)
        image_blur = blur(image)

        try:
            image_noise.save(f'{imagenet_noise_dir}/val/{image_class}/{path}')
        except OSError:
            image_noise = image_noise.convert('RGB')
            image_noise.save(f'{imagenet_noise_dir}/val/{image_class}/{path}')

        try:
            image_invert.save(f'{imagenet_invert_dir}/val/{image_class}/{path}')
        except OSError:
            image_invert = image_invert.convert('RGB')
            image_invert.save(f'{imagenet_invert_dir}/val/{image_class}/{path}')

        try:
            image_blur.save(f'{imagenet_blur_dir}/val/{image_class}/{path}')
        except OSError:
            image_blur = image_blur.convert('RGB')
            image_blur.save(f'{imagenet_blur_dir}/val/{image_class}/{path}')